## This notebook 
- loads Christoffer's gapless layer boundaries
- combines "layer 2" and "layer 3" into the conventional L2/3
- squares off the ends of the dataset to make sure all cells are within VISp
- loads the two combined cell mapping results
- corrects their `(x,y)` coordinates into microns
- saves new files with the subset of cells within VISp



In [5]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point,LineString,  mapping


In [3]:
import json
import geojson


## Load a cell x gene table

In [7]:
# load one cellxgene table
all_cells_geom_mean = pd.read_csv("/Users/brian/work/data/data/cellxgene/Allen_MERFISH_df_geom_mean.csv")
all_cells_neg_weight = pd.read_csv("/Users/brian/work/data/data/cellxgene/Allen_MERFISH_df_neg_weight.csv")


In [9]:
# fix the spatial coordinates- somehow the physical coordinates got lost in Viktor's mapping?
all_cells_geom_mean["x_um"] = all_cells_geom_mean["x"]*0.1078+ 1154.
all_cells_geom_mean["y_um"] = all_cells_geom_mean["y"]*0.1078+ 4548.


all_cells_neg_weight["x_um"] = all_cells_neg_weight["x"]*0.1078+ 1154.
all_cells_neg_weight["y_um"] = all_cells_neg_weight["y"]*0.1078+ 4548.


## Load Christoffer's annotations

In [74]:
with open("/Users/brian/work/data/data/annotations/MERFISH_layer_annotation_7_layers_no_gap.json",'r') as r:
    cml_annotations = geojson.load(r)
# these need to be converted to microns as well:
for geo in cml_annotations:
    shape_coordinates = np.array(geo["coordinates"][0])
    shape_coordinates[:,0] = shape_coordinates[:,0]*0.1078+ 1154.
    shape_coordinates[:,1] = shape_coordinates[:,1]*0.1078+ 4548.
    geo["coordinates"][0] = [[shape_coordinates[i,0],shape_coordinates[i,1]] for i in range(shape_coordinates.shape[0])]
    
# and save revised version to disk:

with open("/Users/brian/work/data/data/annotations/MERFISH_layer_annotation_7_layers_no_gap_um.json", 'w') as w:
    geojson.dump(cml_annotations, w)

## Make a napari viewer...

In [6]:
v = napari.Viewer()

In [28]:
# add cell locations
cell_plots = v.add_points(all_cells_geom_mean[["x_um","y_um"]], symbol='disc',
                                 name = "all_cells_geom_mean",edge_color= [0,0,0,0], size = 10.5,
                                 face_color= 'b' , blending = "translucent", opacity = 0.8)

In [29]:
# Christoffer's annotations.
# note that these need to be converted to microns as well
for geo in cml_annotations:
    shape_coordinates = np.array(geo["coordinates"][0])

    v.add_shapes([shape_coordinates] ,
        shape_type='polygon',
        edge_width=1,
        edge_color='coral',
        face_color='royalblue',
        name=geo["name"])

In [30]:
# Brian's annotations are generally in agreement on the layers, but limited the data to within VISp
with open("/Users/brian/Allen_MERFISH_annotations_geo.json",'r') as r:
    shapejson = geojson.load(r)
    
#last element in the geojson file is the column axis, so ignore it for now
layer_annotations = shapejson["geometries"][:-1]

In [32]:
layer_annotations

[{"coordinates": [[[2524.861858, 6576.111949], [1658.000542, 5844.070207], [1559.125931, 5745.776194], [2149.632232, 4876.052856], [2299.438471, 4923.428033], [3273.661058, 5452.419069], [2851.624504, 6284.883774], [2851.624504, 6284.883774], [2524.861858, 6576.111949]]], "name": "VISp", "type": "Polygon"},
 {"coordinates": [[[2886.383551, 6001.306753], [2658.741191, 6301.073425], [2726.357734, 6283.042347], [2839.051971, 6132.032068], [2839.051971, 6132.032068], [2886.383551, 6001.306753]]], "name": "VISp_I", "type": "Polygon"},
 {"coordinates": [[[2457.148294, 6515.509647], [2614.174367, 6329.851777], [2693.358861, 6242.475784], [2974.600339, 5917.54631], [3165.735324, 5491.588343], [3191.036316, 5415.620914], [2874.087247, 5243.940168], [2843.536349, 5325.027856], [2709.741859, 5636.304831], [2600.521868, 5865.666814], [2499.493376, 6059.532298], [2261.939894, 6288.894281], [2226.982898, 6323.076284], [2457.148294, 6515.509647]]], "name": "VISp_II/III", "type": "Polygon"},
 {"coordi

In [31]:
for geo in layer_annotations:
    shape_coordinates = np.array(geo["coordinates"][0])
    v.add_shapes([shape_coordinates] ,
        shape_type='polygon',
        edge_width=1,
        edge_color='coral',
        face_color='royalblue',
        name=geo["name"])

## use `shapely` to assign regions to cells

In [71]:
# we need to label all the cells with Christoffer's layers (except the 2/3 thing) but cut off the region boundaries according to Brian's VISp boundary:

# first identify those inside Brian's VISp boundary
cell_is_in_layer = {p["name"]:[Polygon(p["coordinates"][0]).intersects(Point(a)) 
                               for a  in all_cells_geom_mean[["x_um","y_um"]].values] for ii,p in enumerate(layer_annotations) }


all_cells_geom_mean["layer"] = "outside_VISp"
for k in cell_is_in_layer.keys():
    all_cells_geom_mean.loc[cell_is_in_layer[k],["layer"]] = k

# now assign layer values based on Christoffer's layers:
cell_is_in_layer = {p["name"]:[Polygon(p["coordinates"][0]).intersects(Point(a[0:2])) and a[2] not in "outside_VISp"
                               for a  in all_cells_geom_mean[["x_um","y_um","layer"]].values] for ii,p in enumerate(cml_annotations) }

for k in cell_is_in_layer.keys():
    if "L2" in k or "L3" in k:
        
        all_cells_geom_mean.loc[cell_is_in_layer[k],["layer"]] = "L2/3"
    else:
        all_cells_geom_mean.loc[cell_is_in_layer[k],["layer"]] = k
        
        
        
# eliminate cells outside VISp:
all_cells_geom_mean = all_cells_geom_mean[all_cells_geom_mean["layer"] != "outside_VISp"]


In [72]:
## repeat for the other combined annotation result :

# first identify those inside Brian's VISp boundary
cell_is_in_layer_neg_weight = {p["name"]:[Polygon(p["coordinates"][0]).intersects(Point(a)) 
                               for a  in all_cells_neg_weight[["x_um","y_um"]].values] for ii,p in enumerate(layer_annotations) }


all_cells_neg_weight["layer"] = "outside_VISp"
for k in cell_is_in_layer_neg_weight.keys():
    all_cells_neg_weight.loc[cell_is_in_layer_neg_weight[k],["layer"]] = k

# now assign layer values based on Christoffer's layers:
cell_is_in_layer_neg_weight = {p["name"]:[Polygon(p["coordinates"][0]).intersects(Point(a[0:2])) and a[2] not in "outside_VISp"
                               for a  in all_cells_neg_weight[["x_um","y_um","layer"]].values] for ii,p in enumerate(cml_annotations) }

for k in cell_is_in_layer_neg_weight.keys():
    if "L2" in k or "L3" in k:
        
        all_cells_neg_weight.loc[cell_is_in_layer_neg_weight[k],["layer"]] = "L2/3"
    else:
        all_cells_neg_weight.loc[cell_is_in_layer_neg_weight[k],["layer"]] = k
        
        
# eliminate cells outside VISp:
all_cells_neg_weight = all_cells_neg_weight.loc[all_cells_neg_weight["layer"] != "outside_VISp"]


In [73]:
# save these results to .csv files
all_cells_geom_mean.to_csv("/Users/brian/work/data/data/cellxgene/Allen_MERFISH_df_geom_mean_LAYER_ANNOTATIONS.csv", index = False)

all_cells_neg_weight.to_csv("/Users/brian/work/data/data/cellxgene/Allen_MERFISH_df_neg_weight_LAYER_ANNOTATIONS.csv", index = False)

In [ ]:
#